# Control Aprendizaje No Supervisado

*Al finalizar, subir el notebook desarrollado a siveducMD*

Considere los datos adjuntos provenientes de una muestra de canciones de spotify, recolectadas por kaggle.com, que incluye para cada canción 11 características (columnas 4 a la 14). Se trata de analizar si tales características permiten agrupar las canciones según género.  

Entre las características disponibles hay algunas que son calculadas por el propio algoritmo de Spotify, y por lo tanto, tienen cierto grado de subjetividad, como son: **acousticness, danceability, energy, instrumentalness, liveness, speechiness y valence**. 

Por otra parte hay otras variables objetivas como **duration_ms, tempo y  loudness**. Son atributos de una canción que no están sujetos a interpretación del algoritmo. Por ejemplo, la duración es la que tenga la pista y el tempo será el que determine un metrónomo.

Por último se consigna la variable **popularity**, cuyo valor  está basado en datos reales de reproducción, pero ponderada por la cercanía en el tiempo (reproducciones más recientes tienen mayor peso a la hora de calcular la popularidad).

En este control se trata de determinar si las características aquí descritas permiten agrupar adecuadamente las canciones de acuerdo al género a la que pertenecen (primera columna de los datos).


In [4]:
library(ggplot2)
datos <-read.csv("tiposMusica.csv",dec=",",sep=";",header=TRUE)
table(datos[,1])
print(datos[1:10,])


 Classical Electronic       Rock 
        56         39         39 

       genre             artist_name
1  Classical Wolfgang Amadeus Mozart
2  Classical Wolfgang Amadeus Mozart
3  Classical Wolfgang Amadeus Mozart
4  Classical         Frederic Chopin
5  Classical         Frederic Chopin
6  Classical         Frederic Chopin
7  Classical    Ludwig van Beethoven
8  Classical    Ludwig van Beethoven
9  Classical   Johann Sebastian Bach
10 Classical   Johann Sebastian Bach
                                                                 track_name
1                              Grabmusik, K, 42: III, Geliebte Seel' (Live)
2       Requiem in D Minor, K, 626: IIIb, Sequence No, 2, Tuba mirum (Live)
3                                12 Duets (No, 10 Andante) - Violin Version
4                                   Mazurkas, Op, 50: No, 2 in A-Flat Major
5                     M‚lodies Polonaises, Op, 74: No, 9, Melodia (Lamento)
6                        Preludes, Op, 28: No, 12 in G-Sharp Minor (Presto)
7                   Fidelio, Op, 72, Act I: Wenn ich diese T\

**Pregunta 1. (2 puntos)** 

**(i)** Realice un análisis exploratorio de los datos de las 11 características de las canciones (columnas 4 a la 14), incluyendo resúmenes de cada variable, boxplots, correlaciones y  análisis de componentes principales (PCA). 

**(ii)** Interprete sus resultados en la pregunta 1. Indique cual es el interés de utilizar PCA en este conjunto de datos. ¿Es necesario realizar alguna acción de limpieza en los datos antes de continuar con su análisis?

**Pregunta 2. (3 puntos)** 

**(i)** Escoja un método de agrupamiento que estime es adecuado para tratar estos datos. En particular, explique las principales características del método considerado y justifique su elección. 

**(ii)** Desarrolle los agrupamientos correspondientes explicando como escogió los valores de los parámetros utilizados. Itere si es necesario.

**(iii)** Grafique sus resultados utilizando una proyección de los datos en el primer plano de componentes principales. Interprete sus resultados.


**Pregunta 3. La medida Purity** 

Esta es una medida útil para cuantificar la calidad del agrupamiento cuando se conoce la clase de cada observación.

Sea $N_{ij}$ el número de observaciones en el cluster $i$ que pertenecen a la clase $j$ y $N_i = \sum_{j=1}^{C} N_{ij}$  el número total de elementos del cluster $i$. Se define $p_{ij} = \frac{N_{ij}}{N_i}$ la probabilidad empírica de la clase $j$ en el cluster $i$. Se denomina **Purity** del cluster $i$ a:
$$p_i = \max_{j= 1,\cdots, C} p_{ij}$$
y la **Purity** del agrupamiento completo:
$$\text{Purity} = \sum_{i} \frac{N_i}{N} p_i$$
El rango de Purity es $[0,1]$ y cuanto mas cercano a $1$ es mejor el agrupamiento (mas cercano a las clases).

Utilice la medida Purity para establecer la calidad del agrupamiento respecto de su capacidad para predecir el género (clase) de las canciones. Realice una conclusión general de sus resultados.

In [5]:
#implementación de Purity
ClusterPurity <- function(clusters, classes) {
  sum(apply(table(classes, clusters), 2, max)) / length(clusters)
}